In [8]:
import pyopencl as cl

print('\n' + '=' * 60 + '\nOpenCL Platforms and Devices')
# Print each platform on this computer
for platform in cl.get_platforms():
    print('=' * 60)
    print('Platform - Name:  ' + platform.name)
    print('Platform - Vendor:  ' + platform.vendor)
    print('Platform - Version:  ' + platform.version)
    print('Platform - Profile:  ' + platform.profile)
    # Print each device per-platform
    for device in platform.get_devices():
        print('    ' + '-' * 56)
        print('    Device - Name:  ' + device.name)
        print('    Device - Type:  ' + cl.device_type.to_string(device.type))
        print('    Device - Max Clock Speed:  {0} Mhz'.format(device.max_clock_frequency))
        print('    Device - Compute Units:  {0}'.format(device.max_compute_units))
        print('    Device - Local Memory:  {0:.0f} KB'.format(device.local_mem_size/1024.0))
        print('    Device - Constant Memory:  {0:.0f} KB'.format(device.max_constant_buffer_size/1024.0))
        print('    Device - Global Memory: {0:.0f} GB'.format(device.global_mem_size/1073741824.0))
        print('    Device - Max Buffer/Image Size: {0:.0f} MB'.format(device.max_mem_alloc_size/1048576.0))
        print('    Device - Max Work Group Size: {0:.0f}'.format(device.max_work_group_size))
print('\n')


OpenCL Platforms and Devices
Platform - Name:  Apple
Platform - Vendor:  Apple
Platform - Version:  OpenCL 1.2 (Nov  1 2018 22:59:11)
Platform - Profile:  FULL_PROFILE
    --------------------------------------------------------
    Device - Name:  Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
    Device - Type:  CPU
    Device - Max Clock Speed:  2900 Mhz
    Device - Compute Units:  4
    Device - Local Memory:  32 KB
    Device - Constant Memory:  64 KB
    Device - Global Memory: 8 GB
    Device - Max Buffer/Image Size: 2048 MB
    Device - Max Work Group Size: 1024
    --------------------------------------------------------
    Device - Name:  Intel(R) Iris(TM) Graphics 550
    Device - Type:  GPU
    Device - Max Clock Speed:  1050 Mhz
    Device - Compute Units:  48
    Device - Local Memory:  64 KB
    Device - Constant Memory:  64 KB
    Device - Global Memory: 2 GB
    Device - Max Buffer/Image Size: 384 MB
    Device - Max Work Group Size: 256




In [2]:
import pyopencl as cl  # Import the OpenCL GPU computing API
import pyopencl.array as pycl_array  # Import PyOpenCL Array (a Numpy array plus an OpenCL buffer object)
import numpy as np  # Import Numpy number tools

context = cl.create_some_context()  # Initialize the Context
queue = cl.CommandQueue(context)  # Instantiate a Queue

a = pycl_array.to_device(queue, np.random.rand(50000).astype(np.float32))
b = pycl_array.to_device(queue, np.random.rand(50000).astype(np.float32))  
# Create two random pyopencl arrays
c = pycl_array.empty_like(a)  # Create an empty pyopencl destination array

program = cl.Program(context, """
__kernel void sum(__global const float *a, __global const float *b, __global float *c)
{
  int i = get_global_id(0);
  c[i] = a[i] + b[i];
}""").build()  # Create the OpenCL program

program.sum(queue, a.shape, None, a.data, b.data, c.data)  # Enqueue the program for execution and store the result in c

print("a: {}".format(a))
print("b: {}".format(b))
print("c: {}".format(c))  
# Print all three arrays, to show sum() worked

a: [0.02657887 0.20581935 0.8030059  ... 0.10739549 0.43004036 0.02984072]
b: [0.3135244  0.6525915  0.7775094  ... 0.7999829  0.62553287 0.9934734 ]
c: [0.34010327 0.8584109  1.5805153  ... 0.9073784  1.0555732  1.0233141 ]


In [3]:
# Use OpenCL To Add Two Random Arrays (This Way Shows Details)

import pyopencl as cl  # Import the OpenCL GPU computing API
import numpy as np  # Import Np number tools

platform = cl.get_platforms()[0]  # Select the first platform [0]
device = platform.get_devices()[0]  # Select the first device on this platform [0]
context = cl.Context([device])  # Create a context with your device
queue = cl.CommandQueue(context)  # Create a command queue with your context

np_a = np.random.rand(50000).astype(np.float32)  # Create a random np array
np_b = np.random.rand(50000).astype(np.float32)  # Create a random np array
np_c = np.empty_like(np_a)  # Create an empty destination array

cl_a = cl.Buffer(context, cl.mem_flags.COPY_HOST_PTR, hostbuf=np_a)
cl_b = cl.Buffer(context, cl.mem_flags.COPY_HOST_PTR, hostbuf=np_b)
cl_c = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, np_c.nbytes)
# Create three buffers (plans for areas of memory on the device)

kernel = """__kernel void sum(__global float* a, __global float* b, __global float* c)
{
    int i = get_global_id(0);
    c[i] = a[i] + b[i];
}"""  # Create a kernel (a string containing C-like OpenCL device code)

program = cl.Program(context, kernel).build()
# Compile the kernel code into an executable OpenCL program

program.sum(queue, np_a.shape, None, cl_a, cl_b, cl_c)
# Enqueue the program for execution, causing data to be copied to the device
#  - queue: the command queue the program will be sent to
#  - np_a.shape: a tuple of the arrays' dimensions
#  - cl_a, cl_b, cl_c: the memory spaces this program deals with

np_arrays = [np_a, np_b, np_c]
cl_arrays = [cl_a, cl_b, cl_c]

for x in range(3):
    cl.enqueue_copy(queue, cl_arrays[x], np_arrays[x])
queue.finish()
# Copy the data for array c back to the host

for x in np_arrays:
	print(x)
# Print all three host arrays, to show sum() worked

[0.29906932 0.8947599  0.5322717  ... 0.79795766 0.37053096 0.1336597 ]
[0.55305034 0.6977554  0.23891568 ... 0.49413458 0.97878355 0.41676253]
[0. 0. 0. ... 0. 0. 0.]


In [4]:
# Use OpenCL To Add Two Random Arrays (This Way Shows Details)

import pyopencl as cl  # Import the OpenCL GPU computing API
import numpy as np  # Import Np number tools

platform = cl.get_platforms()[0]  # Select the first platform [0]
device = platform.get_devices()[1]  # Select the first device on this platform [0]
context = cl.Context([device])  # Create a context with your device
queue = cl.CommandQueue(context)  # Create a command queue with your context

np_a = np.random.rand(50000).astype(np.float32)  # Create a random np array
np_b = np.random.rand(50000).astype(np.float32)  # Create a random np array
np_c = np.empty_like(np_a)  # Create an empty destination array

cl_a = cl.Buffer(context, cl.mem_flags.COPY_HOST_PTR, hostbuf=np_a)
cl_b = cl.Buffer(context, cl.mem_flags.COPY_HOST_PTR, hostbuf=np_b)
cl_c = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, np_c.nbytes)
# Create three buffers (plans for areas of memory on the device)

kernel = """__kernel void sum(__global float* a, __global float* b, __global float* c)
{
    int i = get_global_id(0);
    c[i] = a[i] + b[i];
}"""  # Create a kernel (a string containing C-like OpenCL device code)

program = cl.Program(context, kernel).build()
# Compile the kernel code into an executable OpenCL program

program.sum(queue, np_a.shape, None, cl_a, cl_b, cl_c)
# Enqueue the program for execution, causing data to be copied to the device
#  - queue: the command queue the program will be sent to
#  - np_a.shape: a tuple of the arrays' dimensions
#  - cl_a, cl_b, cl_c: the memory spaces this program deals with

np_arrays = [np_a, np_b, np_c]
cl_arrays = [cl_a, cl_b, cl_c]

for x in range(3):
    cl.enqueue_copy(queue, cl_arrays[x], np_arrays[x])
queue.finish()
# Copy the data for array c back to the host

for x in np_arrays:
	print(x)
# Print all three host arrays, to show sum() worked

[0.09469098 0.91063225 0.85510343 ... 0.8622474  0.9047812  0.7794772 ]
[0.18540546 0.8189802  0.375872   ... 0.21538188 0.13436179 0.7985478 ]
[0. 0. 0. ... 0. 0. 0.]


In [5]:

# Test the speed of your PyOpenCL program
from time import time  # Import time tools

import pyopencl as cl  # Import the OpenCL GPU computing API
import numpy as np  # Import number tools
 
a = np.random.rand(1000).astype(np.float32)  # Create a random array to add
b = np.random.rand(1000).astype(np.float32)  # Create a random array to add

def cpu_array_sum(a, b):  # Sum two arrays on the CPU
    c_cpu = np.empty_like(a)  # Create the destination array
    cpu_start_time = time()  # Get the CPU start time
    for i in range(1000):
            for j in range(1000):  # 1000 times add each number and store it
                    c_cpu[i] = a[i] + b[i]  # This add operation happens 1,000,000 times XXX
    cpu_end_time = time()  # Get the CPU end time
    print("CPU Time: {0} s".format(cpu_end_time - cpu_start_time))  # Print how long the CPU took
    return c_cpu  # Return the sum of the arrays

def gpu_array_sum(a, b):
    context = cl.create_some_context()  # Initialize the Context
    queue = cl.CommandQueue(context, properties=cl.command_queue_properties.PROFILING_ENABLE)  # Instantiate a Queue with profiling (timing) enabled
    a_buffer = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=a)
    b_buffer = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=b)
    c_buffer = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, b.nbytes)  # Create three buffers (plans for areas of memory on the device)
    program = cl.Program(context, """
    __kernel void sum(__global const float *a, __global const float *b, __global float *c)
    {
        int i = get_global_id(0);
        int j;
        for(j = 0; j < 1000; j++)
        {
            c[i] = a[i] + b[i];
        }
    }""").build()  # Compile the device program
    gpu_start_time = time()  # Get the GPU start time
    event = program.sum(queue, a.shape, None, a_buffer, b_buffer, c_buffer)  # Enqueue the GPU sum program XXX
    event.wait()  # Wait until the event finishes XXX
    elapsed = 1e-9*(event.profile.end - event.profile.start)  # Calculate the time it took to execute the kernel
    print("GPU Kernel Time: {0} s".format(elapsed))  # Print the time it took to execute the kernel
    c_gpu = np.empty_like(a)  # Create an empty array the same size as array a
    cl.enqueue_read_buffer(queue, c_buffer, c_gpu).wait()  # Read back the data from GPU memory into array c_gpu
    gpu_end_time = time()  # Get the GPU end time
    print("GPU Time: {0} s".format(gpu_end_time - gpu_start_time))  # Print the time the GPU program took, including both memory copies
    return c_gpu  # Return the sum of the two arrays

cpu_array_sum(a, b)  # Call the function that sums two arrays on the CPU
gpu_array_sum(a, b)  # Call the function that sums two arrays on the GPU

CPU Time: 0.5267820358276367 s
GPU Kernel Time: 0.000281148 s


AttributeError: module 'pyopencl' has no attribute 'enqueue_read_buffer'

In [7]:

# Use OpenCL To Add Two Random Arrays (Using PyOpenCL Arrays and Elementwise)

import pyopencl as cl  # Import the OpenCL GPU computing API
import pyopencl.array as cl_array  # Import PyOpenCL Array (a Numpy array plus an OpenCL buffer object)
import numpy  # Import Numpy number tools

context = cl.create_some_context()  # Initialize the Context
queue = cl.CommandQueue(context)  # Instantiate a Queue

a = cl_array.to_device(queue, numpy.random.randn(10).astype(numpy.float32))  # Create a random pyopencl array
b = cl_array.to_device(queue, numpy.random.randn(10).astype(numpy.float32))  # Create a random pyopencl array
c = cl_array.empty_like(a)  # Create an empty pyopencl destination array
print(a.shape)
sum = cl.elementwise.ElementwiseKernel(context, "float *a, float *b, float *c", "c[i] = a[i] + b[i]", "sum")
# Create an elementwise kernel object
#  - Arguments: a string formatted as a C argument list
#  - Operation: a snippet of C that carries out the desired map operatino
#  - Name: the fuction name as which the kernel is compiled

sum(a, b, c)  # Call the elementwise kernel

print("a: {}".format(a))
print("b: {}".format(b))
print("c: {}".format(c))

(10,)


RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz' on 'Apple' at 0xffffffff>:


(options: -I /Volumes/G-DRIVE/openclvenv3/lib/python3.6/site-packages/pyopencl/cl)
(source saved as /var/folders/75/c7jj66b91pj281kdc3hrn5_r0000gn/T/tmpeyjw8rxj.cl)